In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# standard python
import numpy as np
import scipy
from scipy import stats
#import pathlib

import os
# plotting, especially for jupyter notebooks
import matplotlib
#matplotlib.rcParams['text.usetex'] = True # breaks for some endpoint labels
from matplotlib import pyplot as plt
from IPython.display import Image
import pandas as pd
# tensorflow
import tensorflow as tf
#tf.enable_eager_execution() # needed for tf version 1 or it stages operations but does not do them
from tensorflow import keras
from tensorflow.keras import layers, regularizers
tf.keras.backend.clear_session()  # For easy reset of notebook state.
# local routines
#from chemdataprep import load_PDBs,load_countsfromPDB,load_diametersfromPDB,find_chemnames
#from toxmathandler import load_tscores

#checkpoint_path = "/home2/ajgreen4/Read-Across_w_GAN/Models/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

print("tensorflow version",tf.__version__,". Executing eagerly?",tf.executing_eagerly())
print("Number of GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Import Dataset

In [ ]:
import qm7_weightedviews as CP
import scipy.io
qm7_data = scipy.io.loadmat('qm7.mat')
# Print the dimensions of each array
for key in ['X', 'R', 'Z', 'T', 'P']:
    array = qm7_data.get(key)
    if array is not None:
        print(f"Dimensions of {key}: {array.shape}")
    else:
        print(f"{key} not found in the dataset")

In [ ]:
# Get the atomic numbers array
# Z is numpy array that might be multi-dimensional so its better to convert it to one-dimensional array
atomic_numbers = qm7_data['Z']
all_atomic_numbers = atomic_numbers.flatten()

# Find atomic numbers
unique_atomic_numbers = set(all_atomic_numbers)
unique_atomic_numbers.discard(0)  # Remove 0 if it's not a valid atomic number

# Print unique atomic numbers
print("atomic numbers:", unique_atomic_numbers)

In [ ]:
def atomic_number_to_symbol(atomic_number):
    periodic_table = {
        1: 'H', 6: 'C', 7: 'N', 8: 'O', 16: 'S'
    }
    return periodic_table.get(atomic_number, 'Unknown')

# Convert atomic numbers to element symbols
unique_elements = {atomic_number_to_symbol(int(num)) for num in unique_atomic_numbers}


Z = qm7_data['Z']
R = qm7_data['R']
mollist = []

for mol_idx in range(Z.shape[0]):
    molecule = []
    for atom_idx in range(Z.shape[1]):
        atomic_number = Z[mol_idx, atom_idx]
        if atomic_number != 0:
            atom_type = atomic_number_to_symbol(atomic_number)
            # Ensure coordinates are a NumPy array
            coordinates = np.array(R[mol_idx, atom_idx, :])
            molecule.append((atom_type, coordinates))
    mollist.append(molecule)

In [ ]:
def speciesmap(atom_type):
    atom_to_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'S': 16}
    #print(atom_type)
    return np.array([atom_to_number.get(atom_type, 0)])  # Returns 0 if atom type is not recognized

## Full split to get views

In [ ]:
from qm7_weightedviews import load_qm7_data
ws, vs, Natoms, Nviews = load_qm7_data(mollist, speciesmap, setNatoms=None, setNviews=None, carbonbased=False, verbose=0)

In [ ]:
T=qm7_data['T']
T_reshaped = T.reshape((7165,))
print(type(T_reshaped))
T_reshaped.shape
from sklearn.model_selection import train_test_split
Z = qm7_data['Z']
R = qm7_data['R']

# Split the dataset into training and testing sets
#usual routine, training to testing ratio is 80:20
Z_train, Z_test, R_train, R_test, T_train, T_test = train_test_split(Z, R,T_reshaped, test_size=0.2, random_state=42)

def atomic_number_to_symbol(atomic_number):
    periodic_table = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 16: 'S'}
    return periodic_table.get(atomic_number, 'Unknown')
def convert_to_mollist1(Z_data, R_data):
    mollist1 = []
    for mol_idx in range(Z_data.shape[0]):
        molecule = []
        for atom_idx in range(Z_data.shape[1]):
            atomic_number = Z_data[mol_idx, atom_idx]
            if atomic_number != 0:  # Assuming 0 means no atom present
                atom_type = atomic_number_to_symbol(atomic_number)
                coordinates = np.array(R_data[mol_idx, atom_idx, :])
                molecule.append((atom_type, coordinates))
        mollist1.append(molecule)
    return mollist1

# Convert training and testing data to mollist format
qm7_train = convert_to_mollist1(Z_train, R_train)
qm7_test = convert_to_mollist1(Z_test, R_test)
qm7_labels_train = T_train
qm7_labels_test = T_test

#qm7_labels_train = [chemicaldiameter(x) for x in qm7_train]
#qm7_labels_test = [chemicaldiameter(x) for x in qm7_test]

In [ ]:
print(np.min(T_reshaped))
print(np.max(T_reshaped))
print(np.mean(T_reshaped))
print(np.std(T_reshaped))

In [ ]:
plt.plot(T_reshaped)
plt.title("T_reshaped")
plt.show()

In [ ]:
plt.hist(T_reshaped, bins=30)
plt.title("T")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
ws_qm7_train, vs_qm7_train, Natoms_train, Nviews_train = load_qm7_data(qm7_train, speciesmap, setNatoms=None, setNviews=None, carbonbased=False, not_hydrogen= False, heaviest_origin= False, verbose=0)
qm7G_train=[ws_qm7_train, vs_qm7_train]
ws_qm7_test, vs_qm7_test, Natoms_test, Nviews_test = load_qm7_data(qm7_test, speciesmap, setNatoms=None, setNviews=None, carbonbased=False, not_hydrogen= False, heaviest_origin= False, verbose=0)
qm7G_test=[ws_qm7_test, vs_qm7_test]

In [ ]:
print(ws_qm7_train.shape)
print(vs_qm7_train.shape)
print(ws_qm7_test.shape)
print(vs_qm7_test.shape)

## Find broken views

In [ ]:
from qm7_transformercode import small_views
single_atom_qm7_train = small_views(vs_qm7_train)
single_atom_qm7_test = small_views(vs_qm7_test)
print(single_atom_qm7_train.shape)
print(single_atom_qm7_test.shape)

# Add single atom properties

In [ ]:
from qm7_transformercode import atom_properties
from qm7_transformercode import single_atomic_property_switches
from qm7_transformer import get_embeddings

In [ ]:
atom_embeddings_qm7_train = get_embeddings(single_atom_qm7_train, atom_properties, single_atomic_property_switches)
atom_embeddings_qm7_test  = get_embeddings(single_atom_qm7_test,  atom_properties, single_atomic_property_switches)


In [ ]:
print(atom_embeddings_qm7_train.shape)
print(atom_embeddings_qm7_test.shape)
print(atom_embeddings_qm7_train.dtype)
print(atom_embeddings_qm7_test.dtype)

In [ ]:
#atom_embeddings_qm7_train[0][0]


In [ ]:
labelsG_train = qm7_labels_train
labelsG_test = qm7_labels_test
Ntoxicity = 3
ws_train, vs_train = ws_qm7_train, atom_embeddings_qm7_train
ws_test, vs_test = ws_qm7_test, atom_embeddings_qm7_test
dataG_train=[ws_train,vs_train]
dataG_test=[ws_test,vs_test]

In [ ]:
print(type(labelsG_train))
print(type(labelsG_test))
print(ws_train.shape)
print(vs_train.shape)
print(ws_test.shape)
print(vs_test.shape)
print(labelsG_train.shape)
print(labelsG_test.shape)
print(len(dataG_train))
print(len(dataG_test))

# Transformer Layer

In [ ]:
from tensorflow.keras.layers import Layer, LayerNormalization, Dense

class AtomAttention(Layer):
    
    def __init__(self, d_k = 32, d_v = 16 , **kwargs):
        
        '''
        single head attention for atom-atom relationship
        d_k = dimention of query and key weight matrices
        d_v = dimension of key weight matrices (could be same as d_k)
        
        '''
        super(AtomAttention, self).__init__(**kwargs)
        
        self.d_k = d_k
        self.d_v = d_v
        self.norm = LayerNormalization(axis = -1, epsilon= 1e-6)
    
    def build(self, input_shape):
        
        # our current input for training is (5732,23,23,16) so (batch, views, atoms, features)
        
        self.batch_size = input_shape[0]
        self.num_views = input_shape[1]
        self.num_atoms = input_shape[2]
        self.feature_dim  = input_shape[3]
        
        
        self.W_q = self.add_weight( name = 'W_q', shape = (self.feature_dim, self.d_k), 
                                   initializer= 'glorot_uniform', trainable= True) # 16*32
        
        self.W_k = self.add_weight(name = 'W_k', shape=(self.feature_dim, self.d_k),
                                   initializer = 'glorot_uniform', trainable = True) #16*32
        
        self.W_v = self.add_weight(name = 'W_v' , shape= (self.feature_dim, self.d_v),
                                   initializer = 'glorot_uniform' , trainable = True) #16*16 (ideally 16*32)
        
        self.W_o = self.add_weight(name = 'W_o', shape = (self.d_v, self.feature_dim), 
                                   initializer= 'glorot_uniform', trainable= True) # 16*16
        
    def call(self, inputs):
        
        '''
        input shape = (5732,23,23,16)
        output shape = (5732,23,23,16)
        reshaped_input = convert batches*views into sequences to find atom-atom attention
        '''
        
        reshaped_input = tf.reshape(inputs, [-1, self.num_atoms, self.feature_dim])
        
        Q = tf.matmul(reshaped_input, self.W_q) #sequence, atoms, d_k
        K = tf.matmul(reshaped_input, self.W_k) # sequence, atoms, d_k
        V = tf.matmul(reshaped_input, self.W_v) # sequence, atoms, d_v
        
        # find attention score
        
        scores = tf.matmul(Q, K, transpose_b= True)
        
        # scaling
        
        scaled_scores = scores/tf.math.sqrt(tf.cast(self.d_k, tf.float32))
        
        # mask rows and columns which exactly zero
        
        row_sums = tf.reduce_sum(scaled_scores, axis = 2)
        col_sums = tf.reduce_sum(scaled_scores, axis = 1)
        mask_rows = tf.equal(row_sums, 0.0)
        mask_cols = tf.equal(col_sums, 0.0)
        
        mask = tf.logical_or( tf.expand_dims(mask_rows, 2), tf.expand_dims(mask_cols,1))
        neg_inf = tf.constant(-1e9, dtype=scaled_scores.dtype)
        scaled_scores = tf.where(mask, neg_inf, scaled_scores)
   
        # softmax
        
        attn_weight = tf.nn.softmax(scaled_scores, axis= -1)
        
        # multiply with V
        
        attn_output = tf.matmul(attn_weight, V) # sequence, atoms, d_v
        
        # project back to feature space
        
        project_output = tf.matmul(attn_output, self.W_o) # sequence, atoms, features
        
        # skip connection
        
        skip_output = project_output + reshaped_input
        
        # layer normalization
        
        normed = self.norm(skip_output)
        
        # get back original shape
        
        batch_size = tf.shape(inputs)[0]
        
        final_output = tf.reshape(normed, [batch_size, self.num_views, self.num_atoms, self.feature_dim])
        
        return final_output
    
# Feedforward layer
    

class FeedForward(Layer):
    """Position-wise feed-forward net with residual + LayerNorm."""
    def __init__(self, d_model=16, d_ff=64, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fc1 = Dense(d_ff, activation='relu', name='ffn_fc1')
        self.fc2 = Dense(d_model, name='ffn_fc2')
        self.norm = LayerNormalization(axis=-1, epsilon=1e-6, name='ffn_norm')

    def call(self, x):
        y = self.fc1(x)
        y = self.fc2(y)
        return self.norm(x + y)

class TransformerBlock(Layer):
    """Single Transformer encoder block: AtomAttention + FeedForward."""
    def __init__(self, d_model=16, d_k=32, d_v=16, d_ff=64, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.attn = AtomAttention(d_k=d_k, d_v=d_v, name='atom_attention')
        self.ffn = FeedForward(d_model=d_model, d_ff=d_ff, name='ffn')

    def call(self, x):
        x = self.attn(x)
        x = self.ffn(x)
        return x
    
        
        

## Neural network code 

In [ ]:
# generic dense NN
def multiDense(Nin,Nout,Nhidden,widthhidden=None,kernel_regularizer=None):
    """Construct a basic NN with some dense layers.
    
    :parameter Nin: The number of inputs
    :type Nin: int
    :parameter Nout: The number of outputs
    :type Nout: int
    :parameter Nhidden: The number of hidden layers.
    :type Nhidden: int
    :parameter widthhidden: The width of each hidden layer.
        If left at None, Nin + Nout will be used.
    :parameter kernel_regularizer: the regularizer to use, such as regularizers.l2(0.001)
    :type kernel_regularizer: tensorflow.keras.regularizers.xxx
    :returns: The NN model
    :rtype: keras.Model
    
    """
    if widthhidden is None:
        widthhidden = Nin + Nout
    x = inputs = keras.Input(shape=(Nin,), name='multiDense_input')
    if kernel_regularizer is not None:
        print("Using regularization")
    for i in range(Nhidden):
        x = layers.Dense(widthhidden, activation='relu', kernel_regularizer=kernel_regularizer,name='dense'+str(i))(x)
#        x = layers.Dense(widthhidden, name='dense'+str(i))(x)
#        x = tf.nn.leaky_relu(x, alpha=0.05)
#    outputs = layers.Dense(Nout, activation='linear',name='multiDense_output')(x)
    outputs = layers.Dense(Nout,name='multiDense_output')(x)
    #outputs = tf.nn.leaky_relu(outputs, alpha=0.05)
    return keras.Model(inputs=inputs, outputs=outputs)#, name='multiDense')
if 1:
    # manual check of multiDense
    mmd = multiDense(368,3,5,15)
    mmd.summary()
    # used to do the weighted sum over views

In [ ]:
def parallelwrapper(Nparallel,basemodel,insteadmax=False):
    """Construct a model that applies a basemodel multiple times and take a weighted sum (or max) of the result.
    
    :parameter Nparallel: The number of times to apply in parallel
    :type Nparallel: int
    :parameter basemodel: a keras.Model inferred to have Nin inputs and Nout outputs.
    :type basemodel: a keras.Model
    :parameter insteadmax: If True, take the max of the results of the basemodel instead of the weighted sum.
        For compatibility, the model is still constructed with weights as inputs, but it ignores them.
    :type insteadmax: Boolean
    :returns: model with inputs shape [(?,Nparallel),(?,Nin,Nparallel)] and outputs shape (?,Nout).
        The first input is the scalar weights in the sum.
    :rtype: keras.Model
    
    Note: We could do a max over the parallel applications instead of or in addition to the weighted sum.
    
    """
    # infer shape of basemodel inputs and outputs
    Nin =  basemodel.inputs[0].shape[1]
    Nout =  basemodel.outputs[0].shape[1]
    
    # Apply basemodel Nparallel times in parallel
    # create main input (?,Nparallel,Nin) 
    parallel_inputs = keras.Input(shape=(Nparallel,Nin), name='parallelwrapper_input0')
    # apply base NN to each parallel slice; outputs (?,Nparallel,Nout)
    if False:
        # original version, stopped working at some tensorflow update
        xb = basemodel(parallel_inputs) # worked in earlier tensorflow
        #xb = tf.map_fn(basemodel,parallel_inputs) # another version that fails
    else:
        # newer version, works but makes summary and graphing cumbersome
        # unstack in the Nparallel directio
        parallel_inputsunstacked = tf.keras.ops.unstack(parallel_inputs, Nparallel, 1)
        # apply base NN to each 
        xbunstacked = [basemodel(x) for x in parallel_inputsunstacked]
        # re-stack
        xb = tf.keras.ops.stack(xbunstacked,axis=1)
    
    # create input scalars for weighted sun (?,Nparallel)
    weight_inputs = keras.Input(shape=(Nparallel,), name='parallelScalars')
    if insteadmax:
        # take max over the Nparallel direction to get (?,1,Nout)
        out = layers.MaxPool1D(pool_size=Nparallel)(xb)
        # reshape to (?,Nout)
        out = layers.Reshape((Nout,))(out)
    else:
        # do a weighted sum over the Nparallel direction to get (?,Nout)
        out = layers.Dot((-2,-1))([xb,weight_inputs])
    
    return keras.Model(inputs=[weight_inputs,parallel_inputs], outputs=out, name='parallelwrapper')
if 1:
    # manual check
    mmd = multiDense(368,3,5,15)
    mpw = parallelwrapper(23,mmd,insteadmax=0)
    mpw.summary()
    # make models
  
   

In [ ]:
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras import regularizers, Model

def init_generator(data, labels,
                   baselayers, Nfeatures, endlayers,
                   base_regularizer=None, end_regularizer=None,
                   num_blocks=3, d_ff=64):
    # 1) Extract dims
    Nviews             = data[0].shape[1]   # 23
    Natoms             = data[1].shape[2]   # 23
    d_model            = data[1].shape[3]   # 16 
    flattened_dim      = Natoms * d_model   # 23*16=368

    # 2) Inputs
    weight_input = Input((Nviews,), name='weight_input')             # (batch,23)
    atom_input   = Input((Nviews, Natoms, d_model), name='atom_input')# (batch,23,23,16)

    # 3) Stacked Transformer blocks
    x = atom_input
    for i in range(num_blocks):
        x = TransformerBlock(
                d_model=d_model,
                d_k=32,
                d_v=d_model,
                d_ff=d_ff,
                name=f'transformer_block_{i}'
            )(x)
        # x is still (batch, 23, 23, 16)

    # 4) Flatten & pool
    
    x = Reshape((Nviews, flattened_dim), name='flatten_views')(x)  # (batch,23,368)
    Gbase = multiDense(flattened_dim, Nfeatures, baselayers,
                       kernel_regularizer=base_regularizer)
    Gpw   = parallelwrapper(Nviews, Gbase, insteadmax=False)
    x     = Gpw([weight_input, x])                                 # (batch,Nfeatures)

    # 5) Final MLP 
    Gft    = multiDense(Nfeatures, 1, endlayers,
                        kernel_regularizer=end_regularizer)
    output = Gft(x)                                                # (batch,1)

    # 6) Assemble & compile
    generator = Model([weight_input, atom_input], output,
                      name='generator_with_attention')
    generator.compile(optimizer='adam', loss='mse')
    return generator



# Hyperparameters
baselayers = 2
base_reg = 0
Nfeatures = 3
endlayers = 2
end_reg = 0

# Regularizers
base_regularizer = regularizers.l2(base_reg) if base_reg else None
end_regularizer = regularizers.l2(end_reg) if end_reg else None

print("Model configuration:")
print(f"baselayers={baselayers}, base_reg={base_reg}, Nfeatures={Nfeatures}, endlayers={endlayers}, end_reg={end_reg}")

# Initialize generator
generator = init_generator(dataG_train, labelsG_train, baselayers, Nfeatures, endlayers,
                          base_regularizer=base_regularizer,
                          end_regularizer=end_regularizer)

# Data preparation
dataG_train = [
    np.array(dataG_train[0], dtype='float32'),  # weights (5732, 23)
    np.array(dataG_train[1], dtype='float32')   # atom features (5732, 23, 23, 16)
]
labelsG_train = np.array(labelsG_train, dtype='float32').reshape(-1, 1)

# Test data
dataG_test = [
    np.array(dataG_test[0], dtype='float32'),  # weights (1433, 23)
    np.array(dataG_test[1], dtype='float32')   # atom features (1433, 23, 23, 16)
]
labelsG_test = np.array(labelsG_test, dtype='float32').reshape(-1, 1)

# Verify dimensions
print("\nFinal data shapes:")
print(f"Train weights: {dataG_train[0].shape}")
print(f"Train atom features: {dataG_train[1].shape}")
print(f"Train labels: {labelsG_train.shape}")
print(f"Test weights: {dataG_test[0].shape}")
print(f"Test atom features: {dataG_test[1].shape}")
print(f"Test labels: {labelsG_test.shape}")



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
%%time
# fit
BATCH_SIZE = 32

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

if 1:
    history = generator.fit(
        dataG_train,
        labelsG_train,
        epochs=50,
        batch_size=BATCH_SIZE,
        validation_data=(dataG_test, labelsG_test),
        callbacks=[early_stop],
        verbose=1
    )
    print("train loss =", generator.evaluate(dataG_train, labelsG_train, verbose=1))
    print("test loss  =", generator.evaluate(dataG_test,  labelsG_test,  verbose=1))
#    generator.save(modelpath + "AG-model-RT.h5")

In [ ]:
predicted_train = generator.predict(dataG_train)
predicted_test = generator.predict(dataG_test)

# Calculate the Mean Absolute Error for the training set
mae_train = np.mean(np.abs(predicted_train - labelsG_train))

# Calculate the Mean Absolute Error for the test set
mae_test = np.mean(np.abs(predicted_test - labelsG_test))

# Print out the MAE for both sets
print("Train Mean Absolute Error: ", mae_train)
print("Test Mean Absolute Error: ", mae_test)